<a href="https://colab.research.google.com/github/Isa-Duque/Automa-o---Projeto-MEI/blob/main/tent30jan26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import zipfile
import os
import urllib.parse
from google.colab import drive, files

# --- 1. CONFIGURAÇÕES ---
DATA_TESTE = "20240130"
# Link de Fevereiro/2024 para garantir que os dados do fim de janeiro estejam presentes
url = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2024-02/Estabelecimentos0.zip"

# --- 2. EXECUÇÃO ---
drive.mount('/content/drive', force_remount=True)

if os.path.exists('lote_final.zip'):
    os.remove('lote_final.zip')

print("📥 Baixando arquivo da Receita...")
os.system(f"wget --no-check-certificate {url} -O lote_final.zip")

if os.path.getsize('lote_final.zip') < 1000:
    print("❌ Erro: O download falhou.")
else:
    leads = []
    colunas_alvo = [0, 1, 2, 4, 5, 10, 11, 18, 19, 20, 21, 22, 27]

    print(f"🔍 Minerando leads para: {DATA_TESTE}...")
    try:
        with zipfile.ZipFile('lote_final.zip', 'r') as z:
            lista_arquivos = z.namelist()
            nome = [f for f in lista_arquivos if 'ESTABELE' in f.upper()][0]
            with z.open(nome) as f:
                chunks = pd.read_csv(f, sep=';', encoding='latin-1', header=None,
                                     dtype=str, chunksize=500000, usecols=colunas_alvo)

                for i, chunk in enumerate(chunks):
                    # Filtro: Situação 02 (Ativa) e Data exata
                    condicao = (chunk[5] == '02') & (chunk[10] == DATA_TESTE)
                    res = chunk[condicao].copy()

                    if not res.empty:
                        res.columns = [
                            'cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'nome_fantasia',
                            'situacao', 'data_inicio', 'cnae_principal', 'cep', 'uf',
                            'municipio', 'ddd1', 'telefone1', 'email'
                        ]
                        leads.append(res)
                        print(f"✅ Bloco {i}: {len(res)} leads encontrados!")

        if leads:
            df_final = pd.concat(leads, ignore_index=True)
            df_final['cnpj_completo'] = df_final['cnpj_basico'] + df_final['cnpj_ordem'] + df_final['cnpj_dv']

            def criar_link(row):
                ddd = str(row['ddd1']).strip().replace('.0', '').replace('nan', '')
                tel = str(row['telefone1']).strip().replace('.0', '').replace('nan', '')
                if not tel or tel == '' or len(tel) < 7: return ""
                # Mensagem personalizada usando seu conhecimento de afiliada Ton
                msg = urllib.parse.quote(f"Olá!      tudo bem?")
                return f"https://wa.me/55{ddd}{tel}?text={msg}"

            df_final['link_whatsapp'] = df_final.apply(criar_link, axis=1)

            colunas_ordenadas = [
                'cnpj_completo', 'nome_fantasia', 'cnae_principal', 'email',
                'ddd1', 'telefone1', 'link_whatsapp', 'uf', 'municipio'
            ]
            df_final = df_final[colunas_ordenadas]

            nome_arq = f"LEADS_FINAL_{DATA_TESTE}.xlsx"
            df_final.to_excel(nome_arq, index=False)
            df_final.to_excel(f"/content/drive/MyDrive/{nome_arq}", index=False)

            print(f"🚀 SUCESSO! {len(df_final)} leads salvos.")
            files.download(nome_arq)
        else:
            print("❌ Nenhum lead encontrado para esta data neste arquivo específico.")

    except Exception as e:
        print(f"❌ Erro ao processar: {e}")

Mounted at /content/drive
📥 Baixando arquivo da Receita...
🔍 Minerando leads para: 20240130...
✅ Bloco 8: 1 leads encontrados!
✅ Bloco 9: 1789 leads encontrados!
✅ Bloco 20: 1 leads encontrados!
✅ Bloco 21: 1784 leads encontrados!
✅ Bloco 33: 1790 leads encontrados!
🚀 SUCESSO! 5365 leads salvos.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import pandas as pd
import requests
import time
import os
import re

# --- CONFIGURAÇÃO ---
ARQUIVO_ENTRADA = 'LEADS_FINAL_20240130.xlsx'
CNAES_ALVO = ['9602501', '9602502', '4772500', '5611201', '5611203']

if not os.path.exists(ARQUIVO_ENTRADA):
    print(f"❌ Arquivo '{ARQUIVO_ENTRADA}' não encontrado na lateral!")
else:
    # Lendo tudo como texto para evitar perda de zeros à esquerda
    df = pd.read_excel(ARQUIVO_ENTRADA, dtype=str)

    def apenas_numeros(txt):
        return re.sub(r'\D', '', str(txt))

    print("🔍 Analisando e filtrando sua base...")

    # Criando colunas auxiliares invisíveis para o filtro funcionar
    # Procura o CNPJ na coluna 0, CNAE na 3 e Telefone na 6 (padrão da sua planilha)
    cnpj_limpo = df.iloc[:, 0].apply(apenas_numeros)
    cnae_limpo = df.iloc[:, 3].apply(apenas_numeros)
    tel_limpo = df.iloc[:, 6].apply(apenas_numeros)

    # Filtro: DDD 19 (aceita 19... ou 5519...) E os CNAEs de Salão/Comércio/Restaurante
    mask_ddd = tel_limpo.str.contains('^19|^5519')
    mask_cnae = cnae_limpo.isin(CNAES_ALVO)

    df_filtrado = df[mask_ddd & mask_cnae].copy()
    total = len(df_filtrado)

    if total == 0:
        print("⚠️ O filtro ainda resultou em 0. Vou tentar um filtro MAIS leve (apenas DDD 19)...")
        df_filtrado = df[mask_ddd].copy()
        total = len(df_filtrado)

    print(f"✅ Sucesso! {total} leads encontrados para processar no DDD 19.")
    print("⏳ Iniciando consultas na Receita Federal. O processo é lento (3/min).")

    # --- LOOP DE PROCESSAMENTO TOTAL ---
    for index, row in df_filtrado.iterrows():
        # Usa o CNPJ da linha atual
        cnpj = apenas_numeros(row.iloc[0])

        try:
            url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
            response = requests.get(url, timeout=15)

            if response.status_code == 200:
                dados = response.json()
                # Salva o nome real (essencial para MEI) e a situação
                df_filtrado.at[index, 'NOME_CORRETO_RECEITA'] = dados.get('nome', 'N/A')
                df_filtrado.at[index, 'STATUS_CNPJ'] = dados.get('situacao', 'N/A')
                print(f"✔️ {index+1}/{total} - {dados.get('nome')[:30]}...")
            elif response.status_code == 429:
                print("⏳ Limite atingido. Aguardando 60 segundos...")
                time.sleep(60)

            # Intervalo obrigatório para a API gratuita não bloquear seu IP
            time.sleep(21)

        except Exception as e:
            continue

    # SALVAR
    nome_final = 'LEADS_TOTAL_FILTRADO.xlsx'
    df_filtrado.to_excel(nome_final, index=False)
    print(f"\n🔥 PRONTO! Baixe o arquivo '{nome_final}' na lateral esquerda.")

🔍 Analisando e filtrando sua base...
⚠️ O filtro ainda resultou em 0. Vou tentar um filtro MAIS leve (apenas DDD 19)...
✅ Sucesso! 160 leads encontrados para processar no DDD 19.
⏳ Iniciando consultas na Receita Federal. O processo é lento (3/min).
✔️ 81/160 - 53.709.145 JESSICA BOSCARIOL D...
✔️ 147/160 - FBF CONFECCOES LTDA...
✔️ 169/160 - SANTINA SUSANA SOATO POVIA...
✔️ 180/160 - J R R DE FIGUEIREDO...
✔️ 184/160 - 53.710.262 REGIANE APARECIDA P...
✔️ 189/160 - 53.710.324 TIAGO COSTA DA SILV...
✔️ 238/160 - RISSI MOTOPARTS LTDA...
✔️ 266/160 - G T BELTRATI LTDA...
✔️ 305/160 - 53.711.637 RAISSA LUENI COSTA ...
✔️ 350/160 - PLAN SISTEMAS LTDA...
✔️ 394/160 - 53.712.658 EDUARDO ALVES DIAS...
✔️ 421/160 - 53.712.947 EDSON ANTONIO DA SI...
✔️ 434/160 - 53.713.088 MAURICIO DE OLIVEIR...
✔️ 463/160 - 53.713.408 GUSTAVO HENRIQUE DA...
✔️ 501/160 - 53.713.812 PEDRO HENRIQUE SANT...
✔️ 514/160 - 53.713.959 TAGIDIS HELENA DE O...
✔️ 520/160 - 53.714.027 JOSE RENIVALDO DA S...
✔️ 522/160 - 53